In [17]:
import os
os.environ['TRANSFORMERS_OFFLINE'] = 'yes'
from transformers import BertTokenizer
import datasets

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
# tokenizer.bos_token = tokenizer.cls_token
# tokenizer.eos_token = tokenizer.sep_token

train_data = datasets.load_dataset("ccdv/cnn_dailymail", split = 'train') #datasets.load_dataset("cnn_dailymail", "3.0.0", split="train")
val_data = datasets.load_dataset("ccdv/cnn_dailymail", split = "validation[:10%]")  # datasets.load_dataset("cnn_dailymail", "3.0.0", split="validation[:10%]")

HTTPError: 405 Client Error: Not Allowed for url: https://huggingface.co/api/models/bert-base-uncased

In [16]:
train_data = datasets.load_dataset("ccdv/cnn_dailymail", split = 'train') #datasets.load_dataset("cnn_dailymail", "3.0.0", split="train")


HTTPError: 405 Client Error: Not Allowed for url: https://huggingface.co/api/datasets/ccdv/cnn_dailymail?full=true

In [8]:
import torch
from torch.utils.data import Dataset, DataLoader, SequentialSampler
import pandas as pd
import numpy as np
import h5py 
import pickle
import os
from tqdm import tqdm

In [9]:
!pwd
work_dir = '~/text_sum/'
data_folder = 'preprocessed/bert_data_cnndm_final/'

aa = torch.load(data_folder + 'cnndm.train.80.bert.pt')

/home/bui.thuy/text_sum


In [6]:
# print(aa)
print(type(aa))
print(aa[0].keys())
print(aa[0]['src'])

<class 'list'>
dict_keys(['src', 'tgt', 'src_sent_labels', 'segs', 'clss', 'src_txt', 'tgt_txt'])
[101, 2260, 29627, 12521, 9765, 1010, 2382, 2258, 2286, 102, 101, 5718, 29627, 2692, 2620, 9765, 1010, 1015, 2089, 2286, 102, 101, 1037, 10563, 2038, 2649, 1996, 17082, 2617, 2014, 2227, 2001, 9157, 4237, 2011, 1037, 3899, 1010, 2975, 2014, 2007, 4568, 6441, 1012, 102, 101, 9318, 5232, 1010, 2385, 1010, 2001, 5873, 1037, 2767, 1005, 2015, 2160, 1999, 8233, 1010, 18285, 1010, 2043, 2016, 2001, 4457, 2011, 1996, 4111, 1011, 1037, 2887, 17712, 6590, 4625, 2007, 1037, 18672, 2102, 22384, 1012, 102, 101, 2016, 2056, 1024, 1036, 2043, 2057, 2288, 2000, 1996, 2160, 102, 101, 1045, 2938, 2006, 2028, 1997, 1996, 8397, 1999, 1996, 2542, 2282, 2021, 2026, 2767, 1998, 2014, 102, 101, 13055, 2409, 2033, 2000, 2693, 2138, 2008, 2001, 1996, 3899, 1005, 2015, 3242, 1012, 102, 101, 13925, 1024, 1996, 17082, 2886, 2187, 2385, 29624, 29100, 29624, 11614, 9318, 5232, 2007, 7570, 14343, 15482, 2271, 13268, 644

In [12]:
import glob

# get all train pt files
train_data_files = glob.glob(data_folder + '*train*.pt')
test_data_files = glob.glob(data_folder + '*test*.pt')
valid_data_files = glob.glob(data_folder + '*valid*.pt')

print(test_data_files)
print(len(train_data_files))
print(valid_data_files)
# merge all pt file into 1 list

['preprocessed/bert_data_cnndm_final/cnndm.test.0.bert.pt', 'preprocessed/bert_data_cnndm_final/cnndm.test.5.bert.pt', 'preprocessed/bert_data_cnndm_final/cnndm.test.1.bert.pt', 'preprocessed/bert_data_cnndm_final/cnndm.test.3.bert.pt', 'preprocessed/bert_data_cnndm_final/cnndm.test.2.bert.pt', 'preprocessed/bert_data_cnndm_final/cnndm.test.4.bert.pt']
144
['preprocessed/bert_data_cnndm_final/cnndm.valid.0.bert.pt', 'preprocessed/bert_data_cnndm_final/cnndm.valid.3.bert.pt', 'preprocessed/bert_data_cnndm_final/cnndm.valid.5.bert.pt', 'preprocessed/bert_data_cnndm_final/cnndm.valid.4.bert.pt', 'preprocessed/bert_data_cnndm_final/cnndm.valid.6.bert.pt', 'preprocessed/bert_data_cnndm_final/cnndm.valid.2.bert.pt', 'preprocessed/bert_data_cnndm_final/cnndm.valid.1.bert.pt']


In [19]:
max_words_per_sample = 512

def merge_pt(pt_files, field, max_words_per_sample = 512):
    out_arr = None # numpy array

    # max_words_per_sample = 0
    n_samples = 0
    for fname in pt_files:
        # print('processing ', fn)
        temp_pt = torch.load(fname)
        n_samples += len(temp_pt)
        # for pt in temp_pt:
        #     max_words_per_sample = max(max_words_per_sample, len(pt[field]))
        #     n_samples += 1
    print(n_samples)

    out_arr = -np.ones(shape = (n_samples, max_words_per_sample), dtype= np.int_)
    i = 0
    for fname in pt_files:
        temp_pt = torch.load(fname)
        for pt in temp_pt:
            if (len(pt[field]) < max_words_per_sample):
                out_arr[i, 0:len(pt[field])] = pt[field]
            else:
                out_arr[i, :] = pt[field][:max_words_per_sample]    
            i+=1
    return out_arr


# def merge_text_pt(pt_files, field):
#   # field = 'src_txt' or 'tgt_txt'
  
#   out_list = []
#   for fn in pt_files:
#     print('processing ', fn)
#     temp_pt = torch.load(fn)
#     for pt in temp_pt:
#       out_list.append(pt[field])
#   return out_list

# merged_src_test = merge_pt(test_data_files, 'src')
# print(merged_src_test)
  


In [24]:
valid_dataset1 = torch.load(train_data_files[0])
valid_data1 = valid_dataset1[0]
print(len(valid_data1['src']))

435


In [26]:
merged_test = merge_pt(train_data_files, 'src')
print(merged_test.shape)
print(merged_test)



287083
(287083, 512)
[[  101  5519  5208 ...    -1    -1    -1]
 [  101  6159 24246 ...    -1    -1    -1]
 [  101  1006 13229 ...  2062 10831  1010]
 ...
 [  101  1036  6102 ...  3955  1012   102]
 [  101  3036  3457 ...  3936  2008  2016]
 [  101  1006 13229 ...  2001  8060  2438]]


In [14]:
!pwd
!ls

/home/bui.thuy/text_sum
bert2bert.ipynb       encoder_decoder2.ipynb  README.md      train.ipynb
bert2bert_trainer.py  preprocessed	      test.ipynb
bert_test.py	      preprocess.ipynb	      train2.ipynb
data.py		      __pycache__	      trainer.ipynb


In [27]:
del merged_test

In [30]:
# merge all numeric fields in pt files
keys = list(aa[0].keys())[:-2] # ['src', 'tgt', 'src_sent_labels', 'segs', 'clss']

print('loading test_data.hdf5')
with h5py.File("./preprocessed/test_data.hdf5", "w") as data_file:
    for k in keys:
        print('processing key ', k)
        merged_test = merge_pt(test_data_files, k)
        data_file.create_dataset(name = k, data=merged_test)
data_file.close()
print('done for test.hdf5')


print('loading valid_data')
with h5py.File("./preprocessed/valid_data.hdf5", "w") as data_file:
    for k in keys:
        print('processing key ', k)
        merged_valid = merge_pt(valid_data_files, k)
        data_file.create_dataset(name = k, data=merged_valid)
data_file.close()
print('done for valid.hdf5')

print('processing train_data')
with h5py.File("./preprocessed/train_data.hdf5", "w") as data_file:
    for k in keys:
        print('processing key ', k)
        merged_train = merge_pt(train_data_files, k)
        data_file.create_dataset(name = k, data=merged_train)
data_file.close()

loading test_data.hdf5
processing key  src
11489
processing key  tgt
11489
processing key  src_sent_labels
11489
processing key  segs
11489
processing key  clss
11489
done for test.hdf5
loading valid_data
processing key  src
13367
processing key  tgt
13367
processing key  src_sent_labels
13367
processing key  segs
13367
processing key  clss
13367
done for valid.hdf5
processing train_data
processing key  src
287083
processing key  tgt
287083
processing key  src_sent_labels
287083
processing key  segs
287083
processing key  clss
287083


In [62]:
import random
data_dir = "./preprocessed/"
train_path = data_dir + 'train_data.hdf5'
test_path = data_dir + 'test_data.hdf5'
# test_nopad_path = data_dir + 'test_data.hdf5'
valid_path = data_dir + 'valid_data.hdf5'

# f = h5py.File(test_path, 'r')
# for key in f.keys():
#     print(key)
    
f = h5py.File(train_path, 'r')
print(f.keys())
    
# f = h5py.File(valid_path, 'r')
# for key in f.keys():
#     print(key)


test_src = f['src']
print(test_src.shape)

idx = random.randint(0, test_src.shape[0]-1)
print(idx, test_src[idx])


<KeysViewHDF5 ['clss', 'segs', 'src', 'src_sent_labels', 'tgt']>
(287083, 512)
270538 [  101  2919  6240  4135 10354  2013  1037  3960  6473  4825  1999  2225
  3448  2730  1037  2280  2110  2375  8571  1998  2014  3129  1010  2429
  2000  1037  9870  6406  2011  2037  2336  1012   102   101  2280 11849
  3448  2732  7474  2979  2185  4793  6445  1999  2285  2262  2096  2014
  3129  1010  7157  1010  2351  4793  6391  1999  2089  2286  2044  2027
  2119  4265  5294 12763  2013  2833 16149  1010  1996  4848  4447  1012
   102   101  1996  3232  1010  2040  2020  2496  2005  3438  2086  1010
  8823  2112  1997  2037  6240  4135 10354 12278  1010  2029  2443  6240
  4135 10354  1010 16137  9072 14629  1010 24665 11431  2100  1010 22953
 21408  3669  1998  1037  4897  1010  2012  1037  3960  6473  4825  1999
 25231  1999  2255  2262  1012   102   101  1996  7954  2001  1996  2197
  2027  2052  2412  2031  2362  1010  2429  2000  1996  9870  1010  2029
  2001  2034  2988  2011  1996 10907  

In [13]:
from data import *

test_dataset = SummarisationDataset(test_nopad_path)
item = test_dataset.__getitem__(0)
print(item['tgt'])

[ 1.0000e+00  6.8160e+03  5.2490e+03  2.4040e+03  2.0060e+03  2.1780e+03
  2.2650e+03  1.9970e+03  1.2242e+04  2.0120e+03  1.3635e+04  2.1200e+03
  3.0000e+00  1.9960e+03  2.4030e+03  2.9624e+04  7.2920e+03  2.3500e+03
  3.4100e+03  2.0010e+03  2.3200e+03  1.4429e+04  2.0050e+03  2.0100e+03
  2.4858e+04  2.1350e+03  3.2670e+03  3.0000e+00  2.0210e+03  1.9960e+03
  2.2800e+03  2.0880e+03  2.0530e+03  1.0150e+03  3.5440e+03  2.0000e+03
  2.0310e+03  2.3570e+03  2.0580e+03  1.0370e+03  2.0470e+03  7.0530e+03
  3.0000e+00  5.2490e+03  2.0010e+03  4.1100e+03  2.0060e+03  4.9500e+03
  5.6130e+03  2.0000e+03  2.1890e+03  2.0960e+03  1.0975e+04  1.8908e+04
  9.3550e+03  3.0000e+00  1.1562e+04  2.1820e+03  2.0050e+03  2.2560e+03
  2.3250e+03  5.9720e+03  1.9244e+04  1.0247e+04  1.9980e+03  1.0238e+04
  2.0000e+00  2.0150e+03  2.6094e+04  2.0074e+04  3.0000e+00  1.5876e+04
  4.8420e+03  2.1020e+03  3.9360e+03  2.0080e+03  1.9407e+04  2.0180e+03
  2.0420e+03  2.6350e+03  1.4667e+04  2.0050e+03  2

In [26]:
import sys
sys.getsizeof(merged_train)

merged_train = merge_pt(train_data_files, 'src')
print('merged_train size in mb', merged_train.nbytes / 1000000)

del merged_train

merged_train size in mb 1175.891968


In [ ]:
train_data_file = './cnn_dailymail/train.csv'
train_data = pd.read_csv(train_data_file)
vocab = {}

for idx in train_data.shape[0]:
    article = train_data.iloc[idx]['article']
    summary = train_data.iloc[idx]['highlights']
    


In [3]:
from transformers import BertTokenizer, BertModel

In [4]:
# try 

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower=True)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [20]:
print(aa[0].keys())

dict_keys(['src', 'tgt', 'src_sent_labels', 'segs', 'clss', 'src_txt', 'tgt_txt'])


In [39]:
sents = aa[0]['src_txt']
token_ids = aa[0]['src']

print(len(token_ids))
# print(token_ids[0])
print(sents[0])
print(sents[1])

unflat = [tokenizer.encode(s) for s in sents]
flat_list = [item for sublist in unflat for item in sublist]
print(flat_list)
print(len(flat_list))

print(token_ids)
print(len(token_ids))

for i in range(len(flat_list)):
    if flat_list[i] != token_ids[i]:
        print(i, token_ids[i], flat_list[i] )
        print(tokenizer.decode(token_ids[i]))
        print(tokenizer.decode(flat_list[i]))

479
a university of iowa student has died nearly three months after a fall in rome in a suspected robbery attack in rome .
andrew mogni , 20 , from glen ellyn , illinois , had only just arrived for a semester program in italy when the incident happened in january .
[101, 1037, 2118, 1997, 5947, 3076, 2038, 2351, 3053, 2093, 2706, 2044, 1037, 2991, 1999, 4199, 1999, 1037, 6878, 13742, 2886, 1999, 4199, 1012, 102, 101, 4080, 9587, 29076, 1010, 2322, 1010, 2013, 8904, 3449, 9644, 1010, 4307, 1010, 2018, 2069, 2074, 3369, 2005, 1037, 13609, 2565, 1999, 3304, 2043, 1996, 5043, 3047, 1999, 2254, 1012, 102, 101, 2002, 2001, 10583, 2067, 2000, 3190, 3081, 2250, 10771, 2006, 2233, 2322, 1010, 2021, 2002, 2351, 2006, 4465, 1012, 102, 101, 4080, 9587, 29076, 1010, 2322, 1010, 2013, 8904, 3449, 9644, 1010, 4307, 1010, 1037, 2118, 1997, 5947, 3076, 2038, 2351, 3053, 2093, 2706, 2044, 1037, 2991, 1999, 4199, 1999, 1037, 6878, 13742, 102, 101, 2002, 2001, 2579, 2000, 1037, 2966, 4322, 1999, 1996, 319

In [23]:
print(aa[0].keys())
for k in aa[0].keys():
    print(k, len(aa[0][k]) )
    print(aa[0][k])

dict_keys(['src', 'tgt', 'src_sent_labels', 'segs', 'clss', 'src_txt', 'tgt_txt'])
src 479
[101, 1037, 2118, 1997, 5947, 3076, 2038, 2351, 3053, 2093, 2706, 2044, 1037, 2991, 1999, 4199, 1999, 1037, 6878, 13742, 2886, 1999, 4199, 1012, 102, 101, 4080, 9587, 29076, 1010, 2322, 1010, 2013, 8904, 3449, 9644, 1010, 4307, 1010, 2018, 2069, 2074, 3369, 2005, 1037, 13609, 2565, 1999, 3304, 2043, 1996, 5043, 3047, 1999, 2254, 1012, 102, 101, 2002, 2001, 10583, 2067, 2000, 3190, 3081, 2250, 10771, 2006, 2233, 2322, 1010, 2021, 2002, 2351, 2006, 4465, 1012, 102, 101, 4080, 9587, 29076, 1010, 2322, 1010, 2013, 8904, 3449, 9644, 1010, 4307, 1010, 1037, 2118, 1997, 5947, 3076, 2038, 2351, 3053, 2093, 2706, 2044, 1037, 2991, 1999, 4199, 1999, 1037, 6878, 13742, 102, 101, 2002, 2001, 2579, 2000, 1037, 2966, 4322, 1999, 1996, 3190, 2181, 1010, 2485, 2000, 2010, 2155, 2188, 1999, 8904, 3449, 9644, 1012, 102, 101, 2002, 2351, 2006, 4465, 2012, 7855, 3986, 2902, 1011, 2966, 19684, 1005, 2015, 2436, 14056